# Feature Generator :: COV

In [21]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.stats import skew, kurtosis, linregress
from matplotlib import pyplot as plt
import multiprocessing
from multiprocessing import Pool
import os
import math

In [22]:
# # Path settings

# path_var = 1 

# if (path_var == 1):
#     filepath = '/home/docboy/Desktop/Option_Pricing/datasets/'

# else:
#     filepath = '/home/sharan/Desktop/Option_Pricing/datasets/'


filepath = '/home/docboy/Desktop/REL/'

In [27]:

clean = pd.read_csv(filepath + 'ttm_filter_param_0.04_REL_2014_2018.csv')
clean = clean.dropna()
filt = pd.read_csv(filepath + 'ttm_filter_param_0.04_REL_2015_2018.csv')
ts_data = pd.read_csv(filepath + 'INDEXVALUE_RELIANCE_2015_2018_combined.csv')


# For NIFTY50

# clean = pd.read_csv(filepath + 'ttm_cleaned_NIFTY50_2014_2018_raw.csv')
# clean = clean.dropna()
# filt = pd.read_csv(filepath + 'ttm_filter_param_0.04_NIFTY50_2015_2018_new.csv')
# ts_data = pd.read_csv(filepath + 'INDEXVALUE_NIFTY50_2015_2018_combined_raw.csv')


In [28]:
filt['TTM']=filt['ttm']

In [29]:
#print(clean)
filt['Date'] = pd.to_datetime(filt['Date'])
filt = filt.sort_values(by='Date')
filt.reset_index(inplace = True, drop = True)
clean['Date'] = pd.to_datetime(clean['Date'])
clean['Expiry'] = pd.to_datetime(clean['Expiry'])
clean = clean.sort_values(by='Date')
clean.reset_index(inplace = True, drop = False)
ts_data['Date'] = pd.to_datetime(ts_data['Date'])  
ts_data = ts_data.sort_values(by='Date')
ts_data.reset_index(inplace = True, drop = False)

In [30]:
len(clean)

8518

In [31]:
def stats(data):
    return np.mean(data),np.std(data)

In [32]:
def spreadEMA(data):
    data['Spread']=data['Close']/data['Open']
    return np.mean(data['Spread'])

def jumpEMA(data):
    data['Jump']=data['Open']/data['Close'].shift(1)
    data=data.loc[1:,:]
    return np.mean(data['Jump'])

In [33]:
def atr_data(ext):
    '''
    This function is defined as 
    atr =  max[abs(High - Low)_t, (High_t - Close_t-1), (Low_t - Close_t-1)]
    '''
    # Function Logic
#     ext = pd.DataFrame()
#     temp = data.loc[np.logical_and(data.loc[:, 'Strike Price'] == strike , data.loc[:, 'Expiry']) == expiry]
#     ext = ext.append(temp, ignore_index = True)

    val1 = ext.loc[:, 'High']
    val2 = ext.loc[:, 'Low']
    val3 = ext.loc[:, 'Close']

    val = pd.DataFrame()
    val.insert(0, "ATR", 0)
    for i in range(1, len(ext)):
        low = val2.loc[i]
        high = val1.loc[i]
        prev_close = val3.loc[i-1]
        a = (high/low)
        
        if(high>prev_close):
            b = (high/prev_close)
        else:
            b = (prev_close/high)
            
        if(low>prev_close):
            c = (low/prev_close)
        else:
            c = (prev_close/low)            

        k = max(a, b, c)
        val.loc[i-1, "ATR"] = k
        
    return np.mean(val)[0]
       

### Option Data Functions

In [34]:
def ret(data, date, OHLC ,mode='log'):
    '''
    This function takes in 2 primary params. The function outputs a vector of simple return data
    corressponding to previous 20 days raw ts data.
    The function can also output the log returns on changing the value of the mode param to 'log'.

    ## Modification :: The function can also output just the raw rows if mode param is set to 'e'

    # Demo func
    ret(df, '31-Jan-2019')
    ret(df, '31-Jan-2019', 'log')
    ret(df, '31-Jan-2019', 'e')
    '''

    if (mode == 'log'): # Func logic if log returns is desired
        val = pd.DataFrame()
        temp = np.log(data[OHLC])-np.log(data[OHLC].shift(1))
        val['LR']=temp[1:]

    if (mode == 'e'): # Func logic to extract the raw data
        k = data.index[data['Date']==date][0]
        val = pd.DataFrame()
        val.insert(0, "Close", 0)
        val.insert(0, "High", 0)
        val.insert(0, "Low", 0)
        val.insert(0, "Date", 0)
        val["Close"] = data.loc[k-19:k, "Close"]
        val['High'] = data.loc[k-19:k, "High"]
        val['Low'] = data.loc[k-19:k, "Low"]
        val['Open'] = data.loc[k-19:k, "Open"]
        val['Date'] = data.loc[k-19:k, "Date"]
    val.reset_index(inplace=True, drop=True)
    return val

### Feature Generator

In [35]:
def gen(ts_data, ttm, atmDate, atmError, percentage, op_data, op_data_jump, strike,spot,opPrice, prevOp):
    
    oi = op_data['Open Int'].iloc[-1]
    #     print(oi)

    del_oi = op_data_jump['Change in OI'].iloc[-1]
    #     print(del_oi)

    change_oi = (del_oi/oi) *100

    ts_m,ts_sd = stats(ts_data['Close'])

    avg_atmError=ts_m/strike

    ts_atr = atr_data(ts_data)

    close_log_return = ret(ts_data, atmDate,'Close',mode = 'log')

    open_log_return = ret(ts_data, atmDate,'Open',mode = 'log')

    high_log_return = ret(ts_data, atmDate,'High',mode = 'log')

    low_log_return = ret(ts_data, atmDate,'Low',mode = 'log')    

    close_log_ret_m, close_log_ret_sd = stats(close_log_return['LR'])

    open_log_ret_m, open_log_ret_sd = stats(open_log_return['LR'])

    high_log_ret_m, high_log_ret_sd = stats(high_log_return['LR'])

    low_log_ret_m, low_log_ret_sd = stats(low_log_return['LR'])

    vcov = pd.DataFrame()

    vcov['close_lr']=close_log_return['LR']

    vcov['open_lr']=open_log_return['LR']

    vcov['high_lr']=high_log_return['LR']

    vcov['low_lr']=low_log_return['LR']

    covMatrix = vcov.cov()

    #     print(covMatrix)

    c=[]

    for i in range(4):
        for j in range(i+1,4):
            #             print(covMatrix.iloc[i,j])
            val = round(covMatrix.iloc[i,j],8)
            if(val>0):
                c.append(math.sqrt(val))
            else:
                c.append(-1*math.sqrt(abs(val)))

    return [atmDate] + [close_log_ret_m, close_log_ret_sd, 
    open_log_ret_m, open_log_ret_sd, high_log_ret_m, high_log_ret_sd, low_log_ret_m, 
    low_log_ret_sd] + c + [change_oi, avg_atmError,ts_atr,prevOp] + [ttm, atmError,strike,spot,opPrice,percentage]

In [36]:
cvList=[]
cov_list=['close_lr','open_lr','high_lr','low_lr']
for cl1 in range(len(cov_list)):
    for cl2 in range(cl1+1,len(cov_list)):
        cvl=cov_list[cl1]+"_"+cov_list[cl2]+"_cov"
        cvList.append(cvl)
# print(cvList)
col_name = ['date'] + ['close_mean','close_sd','open_mean','open_sd','high_mean','high_sd','low_mean','low_sd'] + cvList + ['change_oi','avg_atmError','ts_atr','prevOp'] + ['ttm', 'atmError','strike','spot','opPrice','percentage']

In [37]:
# col_name

In [38]:
len(filt)

8138

In [39]:
filt.reset_index(drop=True,inplace=True)
# filt

In [40]:
count = 0
# percentage = []

feat = []
params=[]
for index,row in filt.iterrows():
    ttm = row['TTM']
    strike=row['Strike Price']
    spot=row['Underlying Value']
    atmDate=row['Date']
    expDate=row['Expiry']
    close = row['Close']
    opPrice = close
    opsTS = clean.loc[np.logical_and(clean['Strike Price'] == strike, clean['Expiry'] == expDate)]
    opsTS=opsTS.drop("index",1)
    opsTS.reset_index(inplace=True, drop=True)
#     print(atmDate)
    ind = opsTS.index[opsTS['Date']==atmDate]#Gives Index of that day itself
    ind=ind[0]
    if (ind > 20):
        window = 20
    elif (ind < 1):
        continue
    else:
        window = ind
    
    percentage = (close/strike)*100
    
    ops=opsTS.loc[ind-window:ind-1,:]
    ops.reset_index(inplace=True, drop=True)
    if(window==20):
        adjWindow=19
    else:
        adjWindow=window
        
    opsJump=opsTS.loc[ind-adjWindow:ind,:]
    opsJump.reset_index(inplace=True, drop=True)
    ts = ret(ts_data, atmDate, '', mode = 'e')
    
    atmError=(spot/strike)
    prevOp=(opsTS.loc[ind-1,'Close']/strike)
    params.append((ts,ttm,atmDate,atmError,percentage,ops,opsJump,strike,spot,opPrice,prevOp))
#     print(opsJump.loc[:,['Open Int','Change in OI']])
#     count = count + 1
#     break
    
pool = Pool()
for res in pool.starmap(gen,params):    
    feat.append(res)
    
pool.close()


/home/docboy/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/docboy/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:78: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
/home/docboy/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7143: RuntimeWarning: Degrees of freedom <= 0 for slice
  baseCov = np.cov(mat.T)
/home/docboy/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2451: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
/home/docboy/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2451: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
/home/docboy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in long_scalars
  if __name__ == '__main__':
/home/doc

In [42]:
results=pd.DataFrame.from_records(feat, columns=col_name)
results = results.dropna()
results

,date,close_mean,close_sd,open_mean,open_sd,high_mean,high_sd,low_mean,low_sd,close_lr_open_lr_cov,...,change_oi,avg_atmError,ts_atr,prevOp,ttm,atmError,strike,spot,opPrice,percentage
10,2015-01-05,-0.006832,0.004182,-0.000734,0.002763,-0.000617,0.005147,-0.005433,0.006168,0.004807,...,60.000000,0.981222,1.016254,0.036833,52,0.973167,900,875.85,28.50,3.166667
11,2015-01-05,-0.006832,0.004182,-0.000734,0.002763,-0.000617,0.005147,-0.005433,0.006168,0.004807,...,22.905682,0.981222,1.016254,0.020833,24,0.973167,900,875.85,13.90,1.544444
12,2015-01-05,-0.006832,0.004182,-0.000734,0.002763,-0.000617,0.005147,-0.005433,0.006168,0.004807,...,-1.244813,1.026860,1.016254,0.048895,24,1.018430,860,875.85,34.00,3.953488
13,2015-01-05,-0.006832,0.004182,-0.000734,0.002763,-0.000617,0.005147,-0.005433,0.006168,0.004807,...,29.864253,1.003523,1.016254,0.033011,24,0.995284,880,875.85,22.30,2.534091
14,2015-01-06,-0.020037,0.018984,-0.006187,0.008036,-0.007177,0.010185,-0.020076,0.021312,0.015086,...,887.815126,1.013198,1.028404,0.039535,23,0.972209,860,836.10,14.50,1.686047
15,2015-01-07,-0.009644,0.024379,-0.014278,0.015646,-0.009483,0.009682,-0.013559,0.021635,-0.013587,...,2.764781,1.009233,1.028090,0.016860,22,0.993372,860,854.30,20.70,2.406977
16,2015-01-07,-0.009644,0.024379,-0.014278,0.015646,-0.009483,0.009682,-0.013559,0.021635,-0.013587,...,-41.347271,0.986295,1.028090,0.025341,22,0.970795,880,854.30,12.65,1.437500
17,2015-01-07,-0.009644,0.024379,-0.014278,0.015646,-0.009483,0.009682,-0.013559,0.021635,-0.013587,...,4.836193,1.033262,1.028090,0.026667,22,1.017024,840,854.30,31.50,3.750000
18,2015-01-07,-0.009644,0.024379,-0.014278,0.015646,-0.009483,0.009682,-0.013559,0.021635,-0.013587,...,2000.000000,0.986295,1.028090,0.049432,50,0.970795,880,854.30,25.80,2.931818
19,2015-01-07,-0.009644,0.024379,-0.014278,0.015646,-0.009483,0.009682,-0.013559,0.021635,-0.013587,...,1.388889,1.033262,1.028090,0.045238,50,1.017024,840,854.30,40.00,4.761905


In [43]:
filepath = '/home/docboy/Desktop/Option_Pricing/datasets/'

results.to_csv(filepath + 'REL_feature_set_COV_STEP_1.csv', index = False)



# results.to_csv(filepath + 'NIFTY50_feature_set_COV_STEP_1.csv', index = False)